<a href="https://colab.research.google.com/github/shubhamgundawarNYU/Big-Data-Project-Group-16/blob/main/misc-datasets-notebooks/Spark_NYPD_Arrest_Data_(Historic).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**BIG DATA PROJECT**

### NYPD Arrest Data (Historic)
Link to Dataset (https://data.cityofnewyork.us/Public-Safety/NYPD-Arrests-Data-Historic-/8h9b-rp9u)

### DATA CLEANING AT SCALE

#### Mounting Google Drive to Google Collab Notebook to Load the Data Set

Make sure you have the dataset in your Google Drive and you mount your drive to the Colab.

The file should be at the following path: `gdrive/My Drive/NYPD_Arrest_Data_Historic.csv`


In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#### Importing required and Necessary Libraries for cleaning the data present in the data set

In [ ]:
import numpy as np
import pandas as pd
import io

In [ ]:
!pip install pyspark


# **Running Pyspark in Colab**

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 2.3.2 with hadoop 2.7, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab. One important note is that if you are new in Spark, it is better to avoid Spark 2.4.0 version since some people have already complained about its compatibility issue with python. 
Follow the steps to install the dependencies:

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark

Now that you installed Spark and Java in Colab, it is time to set the environment path which enables you to run Pyspark in your Colab environment. Set the location of Java and Spark by running the following code:

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

Run a local spark session to test your installation:

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark = SparkSession.builder.getOrCreate()

#### Reading the Data Set CSV File using `spark.read.csv()` Function

In [ ]:
df = spark.read.csv("/content/gdrive/MyDrive/NYPD_Arrest_Data_Historic.csv", inferSchema=True, header =True)

In [ ]:
df.count()

5153369

#### Get Data Type for each column present in the Data Set




In [ ]:
df.printSchema()

root
 |-- ARREST_KEY: integer (nullable = true)
 |-- ARREST_DATE: string (nullable = true)
 |-- PD_CD: integer (nullable = true)
 |-- PD_DESC: string (nullable = true)
 |-- KY_CD: integer (nullable = true)
 |-- OFNS_DESC: string (nullable = true)
 |-- LAW_CODE: string (nullable = true)
 |-- LAW_CAT_CD: string (nullable = true)
 |-- ARREST_BORO: string (nullable = true)
 |-- ARREST_PRECINCT: integer (nullable = true)
 |-- JURISDICTION_CODE: integer (nullable = true)
 |-- AGE_GROUP: string (nullable = true)
 |-- PERP_SEX: string (nullable = true)
 |-- PERP_RACE: string (nullable = true)
 |-- X_COORD_CD: double (nullable = true)
 |-- Y_COORD_CD: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Lon_Lat: string (nullable = true)



#### Outputing the List of Columns in the Data Set

In [ ]:
df.columns

['ARREST_KEY',
 'ARREST_DATE',
 'PD_CD',
 'PD_DESC',
 'KY_CD',
 'OFNS_DESC',
 'LAW_CODE',
 'LAW_CAT_CD',
 'ARREST_BORO',
 'ARREST_PRECINCT',
 'JURISDICTION_CODE',
 'AGE_GROUP',
 'PERP_SEX',
 'PERP_RACE',
 'X_COORD_CD',
 'Y_COORD_CD',
 'Latitude',
 'Longitude',
 'Lon_Lat']

#### Get top 10 rows of the arrests dataframe

In [ ]:
df.show(n=10)

+----------+-----------+-----+--------------------+-----+-----------------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------+----------+----------+------------------+------------------+--------------------+
|ARREST_KEY|ARREST_DATE|PD_CD|             PD_DESC|KY_CD|        OFNS_DESC|  LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|     PERP_RACE|X_COORD_CD|Y_COORD_CD|          Latitude|         Longitude|             Lon_Lat|
+----------+-----------+-----+--------------------+-----+-----------------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------+----------+----------+------------------+------------------+--------------------+
|  32311380| 06/18/2007|  511|CONTROLLED SUBSTA...|  235|  DANGEROUS DRUGS|PL 2200300|         M|          Q|             27|                1|    18-24|       M|         BLACK|      null|      null|              null|     

## We see that the columns `X_COORD_CD`,`Y_COORD_CD`, `Latitude` and `Longitude` convey the same data as `Lon_Lat`.

#### Hence, we drop those 2 columns and keep only the `Lon_Lat` columns in our cleaned dataset.

In [ ]:
df = df.drop('X_COORD_CD','Y_COORD_CD', 'Latitude', 'Longitude')

In [ ]:
df.columns

['ARREST_KEY',
 'ARREST_DATE',
 'PD_CD',
 'PD_DESC',
 'KY_CD',
 'OFNS_DESC',
 'LAW_CODE',
 'LAW_CAT_CD',
 'ARREST_BORO',
 'ARREST_PRECINCT',
 'JURISDICTION_CODE',
 'AGE_GROUP',
 'PERP_SEX',
 'PERP_RACE',
 'Lon_Lat']

#### Removing all the **duplicate** entries

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.count()

5153369

In [ ]:
df.distinct().count()

5153369

#### **Checking** if the arrest key is unique or not

In [ ]:
df.select('ARREST_KEY').distinct().count()

5153369

#### As we can see `ARREST_KEY` is unique.

In [ ]:
df1 = df.groupBy('ARREST_KEY').count().filter("count > 1")
df1.drop('count').count()

0

In [ ]:
df1.sort('ARREST_KEY').show(n = 10)

+----------+-----+
|ARREST_KEY|count|
+----------+-----+
+----------+-----+



#### Check for a particular arrest key `32311380`

In [ ]:
df.filter('ARREST_KEY = 32311380').show()

+----------+-----------+-----+--------------------+-----+---------------+----------+----------+-----------+---------------+-----------------+---------+--------+---------+-------+
|ARREST_KEY|ARREST_DATE|PD_CD|             PD_DESC|KY_CD|      OFNS_DESC|  LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|PERP_RACE|Lon_Lat|
+----------+-----------+-----+--------------------+-----+---------------+----------+----------+-----------+---------------+-----------------+---------+--------+---------+-------+
|  32311380| 06/18/2007|  511|CONTROLLED SUBSTA...|  235|DANGEROUS DRUGS|PL 2200300|         M|          Q|             27|                1|    18-24|       M|    BLACK|   null|
+----------+-----------+-----+--------------------+-----+---------------+----------+----------+-----------+---------------+-----------------+---------+--------+---------+-------+



#### We understand, that arrest key is unique. Thus, the ARREST_KEY column can be used to uniquely identify a record in the arrests dataset.

## Find Count of Null, None, NaN of All DataFrame Columns

In [ ]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

+----------+-----------+-----+-------+-----+---------+--------+----------+-----------+---------------+-----------------+---------+--------+---------+-------+
|ARREST_KEY|ARREST_DATE|PD_CD|PD_DESC|KY_CD|OFNS_DESC|LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|PERP_RACE|Lon_Lat|
+----------+-----------+-----+-------+-----+---------+--------+----------+-----------+---------------+-----------------+---------+--------+---------+-------+
|         0|          0|  284|   9066| 9066|     9066|     196|     18861|          8|              0|               10|       17|       0|        0|      1|
+----------+-----------+-----+-------+-----+---------+--------+----------+-----------+---------------+-----------------+---------+--------+---------+-------+



#### Get top 5 rows where Arrest Date is NaN

In [ ]:
df.where(col('ARREST_DATE').isNull()).show(n=5)

+----------+-----------+-----+-------+-----+---------+--------+----------+-----------+---------------+-----------------+---------+--------+---------+-------+
|ARREST_KEY|ARREST_DATE|PD_CD|PD_DESC|KY_CD|OFNS_DESC|LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|PERP_RACE|Lon_Lat|
+----------+-----------+-----+-------+-----+---------+--------+----------+-----------+---------------+-----------------+---------+--------+---------+-------+
+----------+-----------+-----+-------+-----+---------+--------+----------+-----------+---------------+-----------------+---------+--------+---------+-------+



There are no rows where Arrest Date is not present which is the expected scenario since the arrest would have happened on a particular day.

In [ ]:
df = df.filter(df.ARREST_DATE.isNotNull())

In [ ]:
df.show(100)

+----------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------------+--------------------+
|ARREST_KEY|ARREST_DATE|PD_CD|             PD_DESC|KY_CD|           OFNS_DESC|  LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|           PERP_RACE|             Lon_Lat|
+----------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------------+--------------------+
| 173142170| 01/01/2018|  109|ASSAULT 2,1,UNCLA...|  106|      FELONY ASSAULT|PL 1200501|         F|          K|             83|                0|    25-44|       F|      WHITE HISPANIC|POINT (-73.921367...|
| 173151559| 01/01/2018|  729|FORGERY,ETC.,UNCL...|  113|             FORGERY|PL 1702500|         F|          M|             23|                0|    25-44|       M|   

Check if minimum and maximum values of date and time are valid or not.

1. The minimum value is January 1, 2006.

2. The maximum value should not be greater than December 31, 2020.

In [ ]:
## Minimum value of the column in pyspark
df.agg({'ARREST_DATE': 'min'}).show()

+----------------+
|min(ARREST_DATE)|
+----------------+
|      01/01/2006|
+----------------+



In [ ]:
## Maximum value of the column in pyspark
df.agg({'ARREST_DATE': 'max'}).show()

+----------------+
|max(ARREST_DATE)|
+----------------+
|      12/31/2020|
+----------------+



## TODO: Revisit
**Some basic data quality checks are as below:**
1. Check if there are no garbage values in law category column. The valid values are: 'FELONY', 'VIOLATION', 'MISDEMEANOR'.
Check for the value `I`.
2. Check if there are no misspellings in Borough Name. There should be 5 distinct boroughs: Manhattan, Bronx, Queens, Brooklyn, Staten Island. We implement unique method, in case of misspellings multiple values of the same borough would be returned.
3. Ideally, key code should contain only 3 digits. Implementing a check below to see if there are any invalid values for the key code. 

In [ ]:
df.select('LAW_CAT_CD').distinct().show()

+----------+
|LAW_CAT_CD|
+----------+
|         F|
|      null|
|         M|
|         V|
|         I|
+----------+



In [ ]:
df.groupBy('LAW_CAT_CD').count().show()

+----------+-------+
|LAW_CAT_CD|  count|
+----------+-------+
|         F|1404172|
|         M|3411013|
|         V| 293065|
|         I|  26258|
|      null|  18861|
+----------+-------+



### Checks for Borough Name

In [ ]:
df.select('ARREST_BORO').distinct().show()

+-----------+
|ARREST_BORO|
+-----------+
|          K|
|          Q|
|          B|
|          M|
|          S|
|       null|
+-----------+



We can see there are no invalid values for the Borough names and thus no need for additional data correction for the same.

In [ ]:
df.where(col('ARREST_BORO').isNull()).show()

+----------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------+--------------------+
|ARREST_KEY|ARREST_DATE|PD_CD|             PD_DESC|KY_CD|           OFNS_DESC|  LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|     PERP_RACE|             Lon_Lat|
+----------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------+--------------------+
|  86596544| 09/05/2012|  297|FACILITATION 4, C...|  354|ANTICIPATORY OFFE...|PL 1150001|         M|       null|              5|                0|    18-24|       M|         BLACK|POINT (-74.001238...|
|  61712847| 05/13/2009|  567|MARIJUANA, POSSES...|  235|     DANGEROUS DRUGS|PL 2211001|         M|       null|             34|                0|    18-24|       M|         BLACK|POINT (-73.9

#### Dropping Rows where Borough Name is NULL

In [ ]:
df = df.filter(df.ARREST_BORO.isNotNull())

In [ ]:
df.count()

5153361

In [ ]:
df.filter(df.ARREST_BORO.isNull()).show()

+----------+-----------+-----+-------+-----+---------+--------+----------+-----------+---------------+-----------------+---------+--------+---------+-------+
|ARREST_KEY|ARREST_DATE|PD_CD|PD_DESC|KY_CD|OFNS_DESC|LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|PERP_RACE|Lon_Lat|
+----------+-----------+-----+-------+-----+---------+--------+----------+-----------+---------------+-----------------+---------+--------+---------+-------+
+----------+-----------+-----+-------+-----+---------+--------+----------+-----------+---------------+-----------------+---------+--------+---------+-------+



#### We can see that all the key codes are valid 3-digit numbers

In [ ]:
df.filter((df.KY_CD < 100) | (df.KY_CD > 999)).count()

0

## Defining checks for outliers in age group

In [ ]:
df.select('AGE_GROUP').distinct().show()

+---------+
|AGE_GROUP|
+---------+
|      944|
|      740|
|      940|
|      938|
|      323|
|      895|
|      352|
|      894|
|      951|
|      <18|
|    25-44|
|      928|
|     null|
|      320|
|      912|
|      948|
|      943|
|      935|
|      932|
|      194|
+---------+
only showing top 20 rows



#### There are many invalid age groups like unrealistically high age groups, etc. although no negative values are there.

#### Lets find all the invalid age groups and replace them with `NaN`

In [ ]:
valid_age_groups = ['<18','18-24','25-44','45-64','65+',np.NaN]
df = df.withColumn('AGE_GROUP', when(df.AGE_GROUP.isin(valid_age_groups), df.AGE_GROUP).otherwise(np.NaN))
df.show()

+----------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------------+--------------------+
|ARREST_KEY|ARREST_DATE|PD_CD|             PD_DESC|KY_CD|           OFNS_DESC|  LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|           PERP_RACE|             Lon_Lat|
+----------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------------+--------------------+
| 173142170| 01/01/2018|  109|ASSAULT 2,1,UNCLA...|  106|      FELONY ASSAULT|PL 1200501|         F|          K|             83|                0|    25-44|       F|      WHITE HISPANIC|POINT (-73.921367...|
| 173151559| 01/01/2018|  729|FORGERY,ETC.,UNCL...|  113|             FORGERY|PL 1702500|         F|          M|             23|                0|    25-44|       M|   

In [ ]:
df.select('AGE_GROUP').distinct().show()

+---------+
|AGE_GROUP|
+---------+
|      <18|
|    25-44|
|      65+|
|      NaN|
|    18-24|
|    45-64|
+---------+



### Check for Race Values 

In [ ]:
df.select('PERP_RACE').distinct().show()

+--------------------+
|           PERP_RACE|
+--------------------+
|               WHITE|
|               BLACK|
|AMERICAN INDIAN/A...|
|      BLACK HISPANIC|
|      WHITE HISPANIC|
|             UNKNOWN|
|               OTHER|
|ASIAN / PACIFIC I...|
+--------------------+



#### Replace all `UNKNOWN` values with `NaN`

In [ ]:
from pyspark.sql.functions import regexp_replace

df = df.withColumn("PERP_RACE",
  regexp_replace("PERP_RACE", "UNKNOWN", "NaN"))

In [ ]:
df.show(100)

+----------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------------+--------------------+
|ARREST_KEY|ARREST_DATE|PD_CD|             PD_DESC|KY_CD|           OFNS_DESC|  LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|           PERP_RACE|             Lon_Lat|
+----------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------------+--------------------+
| 173142170| 01/01/2018|  109|ASSAULT 2,1,UNCLA...|  106|      FELONY ASSAULT|PL 1200501|         F|          K|             83|                0|    25-44|       F|      WHITE HISPANIC|POINT (-73.921367...|
| 173151559| 01/01/2018|  729|FORGERY,ETC.,UNCL...|  113|             FORGERY|PL 1702500|         F|          M|             23|                0|    25-44|       M|   

In [ ]:
df.select('PERP_RACE').distinct().show()

+--------------------+
|           PERP_RACE|
+--------------------+
|               WHITE|
|               BLACK|
|AMERICAN INDIAN/A...|
|      BLACK HISPANIC|
|      WHITE HISPANIC|
|                 NaN|
|               OTHER|
|ASIAN / PACIFIC I...|
+--------------------+



### Checks for Perpretrator Sex

In [ ]:
df.select('PERP_SEX').distinct().show()

+--------+
|PERP_SEX|
+--------+
|       F|
|       M|
+--------+



#### Checking values in suspect sex

In [ ]:
df.groupBy('PERP_SEX').count().orderBy('count', ascending=False).show()

+--------+-------+
|PERP_SEX|  count|
+--------+-------+
|       M|4288491|
|       F| 864870|
+--------+-------+



#### Get unique values of offense description in sorted order

In [ ]:
df.select('OFNS_DESC').distinct().orderBy('OFNS_DESC', ascending=True).show()

+--------------------+
|           OFNS_DESC|
+--------------------+
|                null|
|            ABORTION|
| ADMINISTRATIVE CODE|
|ADMINISTRATIVE CODES|
|AGRICULTURE & MRK...|
|ALCOHOLIC BEVERAG...|
|ANTICIPATORY OFFE...|
|               ARSON|
|ASSAULT 3 & RELAT...|
|     BURGLAR'S TOOLS|
|            BURGLARY|
|CHILD ABANDONMENT...|
|CHILD ABANDONMENT...|
|CRIMINAL MISCHIEF...|
|CRIMINAL MISCHIEF...|
|   CRIMINAL TRESPASS|
|     DANGEROUS DRUGS|
|   DANGEROUS WEAPONS|
|  DISORDERLY CONDUCT|
|DISRUPTION OF A R...|
+--------------------+
only showing top 20 rows



#### Getting Total Count of Offense Description

In [ ]:
df.select('OFNS_DESC').distinct().count()

88

In [ ]:
df.groupBy('OFNS_DESC').count().show()

+--------------------+------+
|           OFNS_DESC| count|
+--------------------+------+
|OTHER TRAFFIC INF...|175524|
|ANTICIPATORY OFFE...|   762|
|HOMICIDE-NEGLIGEN...|   165|
|CHILD ABANDONMENT...|   474|
|NEW YORK CITY HEA...|   515|
|POSSESSION OF STO...|149876|
|OTHER OFFENSES RE...|  5075|
|VEHICLE AND TRAFF...|204881|
|KIDNAPPING & RELA...|  2070|
|HOMICIDE-NEGLIGEN...|   116|
|OFF. AGNST PUB OR...| 13324|
|      FELONY ASSAULT|210233|
|ALCOHOLIC BEVERAG...|  5943|
|OFFENSES RELATED ...|  2714|
|CRIMINAL MISCHIEF...| 19596|
|UNLAWFUL POSS. WE...|    63|
|         THEFT-FRAUD|  9265|
|   THEFT OF SERVICES|   604|
|MURDER & NON-NEGL...|  2226|
|            JOSTLING|   470|
+--------------------+------+
only showing top 20 rows



**Map Key Codes with Offense Description**

In [ ]:
key_off_mapping = df.groupBy('KY_CD').agg(collect_set('OFNS_DESC').alias('OFNS_DESCS')).orderBy('KY_CD')
key_off_mapping.show()

+-----+--------------------+
|KY_CD|          OFNS_DESCS|
+-----+--------------------+
| null|                  []|
|  101|[MURDER & NON-NEG...|
|  102|[HOMICIDE-NEGLIGE...|
|  103|[HOMICIDE-NEGLIGE...|
|  104|              [RAPE]|
|  105|           [ROBBERY]|
|  106|    [FELONY ASSAULT]|
|  107|          [BURGLARY]|
|  109|     [GRAND LARCENY]|
|  110|[GRAND LARCENY OF...|
|  111|[POSSESSION OF ST...|
|  112|       [THEFT-FRAUD]|
|  113|           [FORGERY]|
|  114|             [ARSON]|
|  115|[PROSTITUTION & R...|
|  116|        [SEX CRIMES]|
|  117|   [DANGEROUS DRUGS]|
|  118| [DANGEROUS WEAPONS]|
|  119|[INTOXICATED/IMPA...|
|  120|[ENDAN WELFARE IN...|
+-----+--------------------+
only showing top 20 rows



In [ ]:
key_off_mapping.count()

76

In [ ]:
df.select('KY_CD').distinct().count()

76

#### Each key code represents a particular offense description. There is a one to one mapping. So we would use key code for future analysis instead of offense description.

#### Calculating the null values present in the data columnwise (with respect to the features)

In [ ]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----------+-----------+-----+-------+-----+---------+--------+----------+-----------+---------------+-----------------+---------+--------+---------+-------+
|ARREST_KEY|ARREST_DATE|PD_CD|PD_DESC|KY_CD|OFNS_DESC|LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|PERP_RACE|Lon_Lat|
+----------+-----------+-----+-------+-----+---------+--------+----------+-----------+---------------+-----------------+---------+--------+---------+-------+
|         0|          0|  284|   9066| 9066|     9066|     196|     18861|          0|              0|               10|      195|       0|    51000|      1|
+----------+-----------+-----+-------+-----+---------+--------+----------+-----------+---------------+-----------------+---------+--------+---------+-------+



In [ ]:
amount_missing_df = df.select([(count(when(isnan(c) | col(c).isNull(), c))/count(lit(1))).alias(c) for c in df.columns])
amount_missing_df.show()

+----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+---------------+--------------------+--------------------+--------+--------------------+--------------------+
|ARREST_KEY|ARREST_DATE|               PD_CD|             PD_DESC|               KY_CD|           OFNS_DESC|            LAW_CODE|          LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|   JURISDICTION_CODE|           AGE_GROUP|PERP_SEX|           PERP_RACE|             Lon_Lat|
+----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+---------------+--------------------+--------------------+--------+--------------------+--------------------+
|       0.0|        0.0|5.510966532327155E-5|0.001759240231763...|0.001759240231763...|0.001759240231763...|3.803343099775079E-5|0.003659941541064...|        0.0|            0.0|1.9404

#### Thus, we can see that the percentage of null values per variable has gone considerably down after cleaning. Some variables like 'PARKS_NM', 'HADEVELOPT' and such can have null values as established above. 

### Number of columns in Clean Data

In [ ]:
len(df.columns)

15

### Number of rows in Clean Data

In [ ]:
df.count()

5153361

In [ ]:
df.printSchema()

root
 |-- ARREST_KEY: integer (nullable = true)
 |-- ARREST_DATE: string (nullable = true)
 |-- PD_CD: integer (nullable = true)
 |-- PD_DESC: string (nullable = true)
 |-- KY_CD: integer (nullable = true)
 |-- OFNS_DESC: string (nullable = true)
 |-- LAW_CODE: string (nullable = true)
 |-- LAW_CAT_CD: string (nullable = true)
 |-- ARREST_BORO: string (nullable = true)
 |-- ARREST_PRECINCT: integer (nullable = true)
 |-- JURISDICTION_CODE: integer (nullable = true)
 |-- AGE_GROUP: string (nullable = true)
 |-- PERP_SEX: string (nullable = true)
 |-- PERP_RACE: string (nullable = true)
 |-- Lon_Lat: string (nullable = true)



### **Exporting Clean Data in CSV**

The Cleaned Data Set will be saved as `NYPD_Complaint_Data_Historic_Cleaned.csv`

In [ ]:
pd_df = df.toPandas()
pd_df.to_csv("NYPD_Arrest_Data_Year_to_Date_Cleaned_Spark.csv")